In [1]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import torch.nn.functional as F
import nbimporter
from network import *
import os
from utils import *
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib

In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
num_class=2
classifier_out_dim=2
num_epoch=2500
test_inverval = 50
num_view = 3
dim_hvcdn= 64
batch_size = 153

In [4]:
# 读取制表符分隔的CSV文件
Methylation_train = pd.read_csv('../../DATA/BRCA_survival/1_tr.csv')
# 显示 DataFrame
Methylation_train

,0.4388849177984275,0.0971396987702086,0.1464783281733745,0.3202959830866808,0.7835778781038374,0.1506017191977076,0.1820076415422022,0.1053309296935359,0.1597501115573404,0.7057069269350684,...,0.1938997821350765,0.7852328890220159,0.3521194605009633,0.2169369369369371,0.8426470588235293,0.228607594936709,0.2920901458241273,0.7171229200214708.1,0.82016210739615,0.1997296383913485
0,0.679056,0.018654,0.016060,0.593929,0.876552,0.129513,0.020493,0.895957,0.035029,0.925173,...,0.019608,0.926764,0.204239,0.061622,0.662500,0.131329,0.093239,0.856414,0.959642,0.050017
1,0.195497,0.013818,0.013932,0.363938,0.870485,0.078510,0.017714,0.860160,0.055556,0.924495,...,0.018736,0.850831,0.661609,0.054414,0.588971,0.089557,0.067167,0.769726,0.938197,0.047651
2,0.174768,0.067431,0.038313,0.505738,0.906744,0.247278,0.010073,0.797837,0.043061,0.853192,...,0.010022,0.832560,0.714355,0.061622,0.584559,0.206646,0.162174,0.851047,0.852246,0.014194
3,0.029307,0.121597,0.027090,0.754304,0.427342,0.080229,0.148663,0.929951,0.130745,0.877999,...,0.223529,0.927812,0.732418,0.250450,0.775368,0.077848,0.185815,0.779388,0.941743,0.278472
4,0.308077,0.016996,0.112423,0.503624,0.746896,0.320344,0.012852,0.660572,0.063588,0.750983,...,0.018301,0.829115,0.653661,0.048649,0.645221,0.362975,0.463102,0.769189,0.821851,0.081784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,0.263760,0.016858,0.060759,0.269405,0.776947,0.311748,0.007642,0.597476,0.033021,0.790565,...,0.006100,0.926913,0.658478,0.120360,0.605515,0.325000,0.321918,0.530864,0.842621,0.108145
454,0.071122,0.022109,0.021478,0.662036,0.903499,0.107736,0.013546,0.789853,0.055332,0.926122,...,0.003050,0.682342,0.857418,0.008288,0.758456,0.254114,0.156209,0.900429,0.938703,0.041230
455,0.020729,0.388697,0.043537,0.532770,0.927624,0.333238,0.281348,0.875612,0.575636,0.869595,...,0.478867,0.906096,0.798651,0.425225,0.377206,0.415190,0.709898,0.964842,0.946471,0.412301
456,0.080415,0.244576,0.062113,0.289792,0.962331,0.124642,0.120875,0.916044,0.129630,0.880710,...,0.661438,0.881084,0.588150,0.453333,0.738971,0.473418,0.144057,0.775899,0.927558,0.447786


In [5]:
# 读取制表符分隔的CSV文件
Methylation_test = pd.read_csv('../../DATA/BRCA_survival/1_te.csv')
# 显示 DataFrame
Methylation_test

,0.6072194424588993,0.1258808898714937,0.1484133126934984,0.2156448202959832,0.774971783295711,0.5249283667621776,0.168808614102119,0.6294102498068503,0.0680499776885317,0.7741629388640369,...,0.4535947712418302,0.3741201138235735,0.3660886319845857,0.3030630630630631,0.4352941176470588,0.5389240506329114,0.7631462660185595,0.5974235104669887.1,0.7700101317122594,0.1841838458938831
0,0.626162,0.013680,0.013545,0.482785,0.971924,0.114040,0.014936,0.952614,0.019857,0.968822,...,0.010458,0.703460,0.381262,0.057658,0.629044,0.075949,0.070482,0.833333,0.944107,0.037513
1,0.040386,0.242089,0.036765,0.886741,0.036964,0.012321,0.215353,0.858357,0.180946,0.965162,...,0.572985,0.797214,0.783237,0.420901,0.749265,0.152532,0.252983,0.918948,0.977710,0.427509
2,0.039671,0.120492,0.041602,0.311235,0.851157,0.177650,0.026745,0.840845,0.170683,0.838688,...,0.095425,0.743747,0.683767,0.134775,0.350735,0.156962,0.250994,0.505099,0.909659,0.288273
3,0.443531,0.168716,0.073297,0.471912,0.663318,0.308138,0.077457,0.136750,0.126952,0.727504,...,0.132898,0.775348,0.548892,0.449369,0.730882,0.305380,0.318294,0.350778,0.461837,0.156472
4,0.283059,0.006494,0.054954,0.301873,0.895034,0.194269,0.016325,0.881535,0.071843,0.936424,...,0.016558,0.737008,0.635597,0.098739,0.748162,0.188291,0.242156,0.918680,0.971125,0.104765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.586133,0.101009,0.265867,0.211265,0.485609,0.743266,0.302883,0.166109,0.104641,0.481497,...,0.397386,0.558784,0.288054,0.295135,0.488603,0.693987,0.776845,0.385132,0.381966,0.339642
148,0.049678,0.068813,0.020704,0.270160,0.496473,0.046132,0.109413,0.903940,0.077644,0.474448,...,0.192157,0.450651,0.768304,0.244685,0.705515,0.054747,0.240168,0.772410,0.952381,0.090909
149,0.012152,0.267100,0.022059,0.448203,0.918030,0.044126,0.179576,0.837239,0.111780,0.861326,...,0.474946,0.824023,0.672447,0.365405,0.653676,0.099684,0.139638,0.750671,0.885849,0.381210
150,0.072909,0.023490,0.052632,0.526276,0.707534,0.362464,0.016672,0.807881,0.062472,0.897248,...,0.002614,0.685338,0.754094,0.073874,0.652206,0.569620,0.238400,0.772410,0.944613,0.087192


In [6]:
# # 读取制表符分隔的CSV文件
# Methylation_val = pd.read_csv('../../DATA/BRCA_survival/methylation_val.csv')
# # 显示 DataFrame
# Methylation_val

In [7]:
# 读取制表符分隔的CSV文件
miRNASeq_train = pd.read_csv('../../DATA/BRCA_survival/3_tr.csv')
# 显示 DataFrame
miRNASeq_train

,0.4170409817087287,0.359123085618927,0.4652716142099635,0.3541872988363455,0.5325353012958335,0.2918074834037416,0.4241145709531734,0.6050296627810159,0.3337271785575554,0.5429385588258255,...,0.3857058823529412,0.2456376192735514,0.3108428254648143,0.3093764280669686,0.328096769929053,0.3393195249297149,0.2900004403152657,0.5289970599196654,0.5720433841065049,0.0944722685958078
0,0.424763,0.370401,0.333037,0.517269,0.503760,0.281518,0.527068,0.523808,0.315260,0.607216,...,0.154618,0.266540,0.449576,0.373825,0.456782,0.451718,0.161094,0.378225,0.567036,0.000000
1,0.546620,0.525651,0.517967,0.528952,0.498693,0.247646,0.599744,0.711894,0.407436,0.734184,...,0.294618,0.399889,0.532608,0.327531,0.573646,0.391474,0.354806,0.696157,0.714321,0.035819
2,0.420109,0.347173,0.476605,0.517362,0.411726,0.798386,0.457051,0.652568,0.420028,0.639798,...,0.418412,0.439269,0.543657,0.359367,0.494348,0.000000,0.483290,0.532331,0.680686,0.336858
3,0.346064,0.462331,0.737512,0.448549,0.643640,0.376992,0.423505,0.683701,0.351159,0.540505,...,0.461647,0.567531,0.276538,0.167449,0.330454,0.418584,0.307499,0.466469,0.536262,0.000000
4,0.460963,0.347926,0.282492,0.397948,0.624754,0.414922,0.617468,0.524563,0.392416,0.527962,...,0.387382,0.490906,0.405701,0.331741,0.445071,0.395175,0.089895,0.572508,0.572144,0.027101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,0.558740,0.646987,0.325095,0.733242,0.441002,0.381005,0.685257,0.600151,0.483329,0.569365,...,0.277500,0.434841,0.422739,0.559622,0.479179,0.523065,0.447994,0.584507,0.590090,0.000000
454,0.513834,0.480997,0.479791,0.399000,0.491510,0.325000,0.558890,0.579777,0.453758,0.535431,...,0.145147,0.379250,0.359115,0.254954,0.486813,0.000000,0.380758,0.601640,0.618900,0.000000
455,0.272829,0.838667,0.880157,0.537881,0.996920,0.810969,0.500371,0.602506,0.301453,0.433823,...,0.547794,0.618008,0.502075,0.125988,0.267847,0.000000,0.292272,0.355211,0.485741,0.491393
456,0.549282,0.646368,0.587276,0.716839,0.485897,0.682031,0.697751,0.770035,0.566166,0.691766,...,0.511765,0.558306,0.651384,0.642059,0.635053,0.436686,0.659636,0.585200,0.757595,0.176813


In [8]:
# 读取制表符分隔的CSV文件
miRNASeq_test = pd.read_csv('../../DATA/BRCA_survival/3_te.csv')
# 显示 DataFrame
miRNASeq_test

,0.1176082426487613,0.0,0.2269748071277395,0.2412416439712801,0.0912197106165378,0.5104254677127338,0.0.1,0.1521908825978353,0.1751909879580474,0.2727118952819632,...,0.3183529411764706,0.3172808266118401,0.2419119458711597,0.1928461634331768,0.4037368351474023,0.1411440702277813,0.4453436660649024,0.2916373348792911,0.4124779561590263,0.2477462437395659
0,0.441943,0.572551,0.350845,0.442498,0.455483,0.465932,0.567640,0.541632,0.303800,0.601140,...,0.201412,0.281275,0.283722,0.416615,0.363879,0.446899,0.316261,0.508924,0.511353,0.000000
1,0.056807,0.178909,0.472918,0.448997,0.293800,0.538865,0.373915,0.351504,0.125793,0.334049,...,0.376324,0.472323,0.280007,0.107626,0.140958,0.000000,0.405398,0.256512,0.393719,0.000000
2,0.369264,0.499825,0.546073,0.686417,0.430879,0.499019,0.360382,0.801572,0.259258,0.715122,...,0.348206,0.242185,0.399276,0.391744,0.500991,0.000000,0.402924,0.493613,0.676995,0.151771
3,0.517284,0.455724,0.306775,0.467582,0.338625,0.616611,0.516461,0.661558,0.350285,0.636688,...,0.231471,0.293848,0.391866,0.271987,0.495681,0.225773,0.346854,0.600781,0.606043,0.036264
4,0.316393,0.249320,0.452914,0.342179,0.333000,0.442517,0.372474,0.461386,0.411272,0.397715,...,0.308941,0.359850,0.174440,0.260189,0.310856,0.126370,0.266021,0.422026,0.489899,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.529648,0.381934,0.229728,0.376578,0.375803,0.329692,0.570968,0.603013,0.323449,0.644612,...,0.378118,0.373926,0.531717,0.401756,0.523840,0.278214,0.292651,0.601277,0.644521,0.125635
148,0.288365,0.137163,0.674518,0.282403,0.177454,0.478787,0.438882,0.478560,0.244772,0.277747,...,0.527941,0.621540,0.000000,0.000000,0.298903,0.199036,0.437612,0.380958,0.480770,0.000000
149,0.417238,0.490997,0.681220,0.474638,0.417165,0.363156,0.282075,0.835697,0.253787,0.652042,...,0.526235,0.461516,0.527927,0.297744,0.557738,0.165385,0.606437,0.559702,0.732312,0.000000
150,0.433712,0.500565,0.590542,0.493207,0.526585,0.508253,0.619488,0.665435,0.522012,0.642790,...,0.348029,0.311587,0.449520,0.434645,0.554620,0.267342,0.334939,0.549557,0.696796,0.044648


In [9]:
# # 读取制表符分隔的CSV文件
# miRNASeq_val = pd.read_csv('../../DATA/BRCA_survival/miRNASeq_val.csv')
# # 显示 DataFrame
# miRNASeq_val

In [10]:
# 读取制表符分隔的CSV文件
RNAseq_train = pd.read_csv('../../DATA/BRCA_survival/2_tr.csv')
# 显示 DataFrame
RNAseq_train

,0.5608555785167986,0.1766963352671975,0.6809032567128923,0.4107881498043606,0.7015425270403147,0.2326074935361899,0.2890965268212788,0.7044968743698328,0.4316677429770745,0.27732987883868,...,0.4884248988646744,0.38787170497338,0.7090433094856572,0.2148936170212763,0.3616304689959078,0.0.21,0.3116487698790267,0.4638785035877668,0.4313665799523991,0.2154454446033482
0,0.604593,0.637959,0.576380,0.556037,0.587082,0.309994,0.495105,0.562674,0.473200,0.381833,...,0.297612,0.527347,0.665463,0.589118,0.408971,0.368389,0.403860,0.473307,0.492971,0.233771
1,0.539083,0.618455,0.657149,0.412409,0.595793,0.387709,0.731452,0.490058,0.442969,0.430115,...,0.421793,0.440878,0.538040,0.485455,0.613818,0.468062,0.462281,0.537074,0.567139,0.315176
2,0.496261,0.364221,0.572076,0.034628,0.327480,0.157582,0.466553,0.712079,0.579371,0.137149,...,0.355005,0.200961,0.931007,0.517893,0.166950,0.043348,0.408264,0.127000,0.258455,0.187488
3,0.520356,0.413814,0.456477,0.515484,0.761538,0.371314,0.432602,0.516757,0.318231,0.420681,...,0.447919,0.498675,0.422939,0.540426,0.533617,0.123905,0.673726,0.541463,0.493690,0.285887
4,0.503568,0.641112,0.720854,0.369704,0.622573,0.345743,0.516175,0.480762,0.387250,0.479982,...,0.474566,0.659278,0.509738,0.446983,0.628140,0.429199,0.419057,0.594408,0.548459,0.295519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,0.518290,0.473217,0.599275,0.311710,0.711283,0.303478,0.551018,0.507965,0.461858,0.307171,...,0.407230,0.248215,0.594732,0.498500,0.514857,0.601895,0.493870,0.649444,0.369486,0.223283
454,0.522572,0.518576,0.670942,0.491336,0.544386,0.373661,0.500875,0.567977,0.410559,0.371348,...,0.369594,0.646799,0.507798,0.361039,0.585143,0.645461,0.290580,0.648794,0.492860,0.192213
455,0.291229,0.204542,0.584862,0.728032,0.665038,0.418048,0.692365,0.365053,0.610510,0.345424,...,0.497534,0.695027,0.185741,0.476352,0.736701,0.207578,0.664320,0.692567,0.776914,0.299501
456,0.622127,0.697371,0.557622,0.552068,0.753733,0.450868,0.765663,0.269570,0.394616,0.452412,...,0.398408,0.476639,0.345212,0.496338,0.753824,0.665729,0.270110,0.510903,0.706592,0.316256


In [11]:
# 读取制表符分隔的CSV文件
RNAseq_test = pd.read_csv('../../DATA/BRCA_survival/2_te.csv')
# 显示 DataFrame
RNAseq_test

,0.8306099405611542,0.1649400021620842,0.8170112246330727,0.0158188932364446,0.6762844149459194,0.5039829026617448,0.5283103153035773,0.5772333131679779,0.4171577332902809,0.3703726282100128,...,0.4583844447344383,0.4175172055577197,0.7479518833645533,0.4747471224276247,0.4240163676424298,0.153780898546099,0.5342938697838793,0.4278382834451846,0.36677367576244,0.2980146378396566
0,0.461578,0.390022,0.469320,0.591140,0.707473,0.352722,0.755612,0.524844,0.421154,0.401219,...,0.389847,0.345059,0.452955,0.573736,0.533805,0.299045,0.600979,0.408262,0.555239,0.286896
1,0.403183,0.205425,0.400519,0.579989,0.652809,0.422277,0.430890,0.368744,0.516387,0.471158,...,0.513089,0.528126,0.512040,0.721904,0.487252,0.300473,0.576893,0.608364,0.611751,0.289195
2,0.515264,0.445074,0.724563,0.510397,0.709731,0.568128,0.645833,0.343154,0.295528,0.507247,...,0.604019,0.616258,0.262565,0.441786,0.550677,0.168521,0.396493,0.612637,0.570737,0.358305
3,0.589105,0.613924,0.777077,0.489296,0.656865,0.420006,0.601124,0.515104,0.437258,0.501715,...,0.262247,0.518803,0.472184,0.398430,0.562764,0.498316,0.493870,0.501126,0.419992,0.347943
4,0.532777,0.115951,0.342066,0.969285,0.544386,0.221862,0.429736,0.645191,0.365172,0.293241,...,0.483727,0.306636,0.616918,0.458982,0.331634,0.129868,0.382058,0.594989,0.524520,0.199938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.595688,0.604086,0.641473,0.489575,0.629932,0.314433,0.553773,0.605182,0.445128,0.404511,...,0.332376,0.467848,0.583945,0.417824,0.550268,0.461040,0.458910,0.526438,0.552527,0.267155
148,0.584567,0.383734,0.501854,0.509419,0.698516,0.405195,0.583516,0.763682,1.000000,0.576743,...,0.479421,0.521621,0.396827,0.495256,0.567391,0.287017,0.489248,0.476325,0.679969,0.307002
149,0.118026,0.269351,0.454222,0.312186,0.647262,0.470805,0.716264,0.227889,0.246489,0.508116,...,0.823829,0.684275,0.253786,0.386118,0.455807,0.093172,0.484600,0.877877,0.933498,0.366913
150,0.480198,0.571223,0.638012,0.438010,0.527532,0.345922,0.381082,0.551482,0.451889,0.368391,...,0.286989,0.416530,0.818750,0.440426,0.512307,0.304001,0.276172,0.412674,0.351968,0.210160


In [12]:
# # 读取制表符分隔的CSV文件
# RNAseq_val = pd.read_csv('../../DATA/BRCA_survival/RNAseq_val.csv')
# # 显示 DataFrame
# RNAseq_val

In [13]:
# 使用np.load()函数加载.npy文件
labels_train = pd.read_csv('../../DATA/BRCA_survival/labels_tr.csv')
labels_train.value_counts()

0
0    315
1    143
Name: count, dtype: int64

In [14]:
# 使用np.load()函数加载.npy文件
labels_test = pd.read_csv('../../DATA/BRCA_survival/labels_te.csv')
labels_test.value_counts()

0
0    107
1     45
Name: count, dtype: int64

In [15]:
# # 使用np.load()函数加载.npy文件
# labels_val = pd.read_csv('../../DATA/BRCA_survival/labels_val.csv')
# labels_val.value_counts()

In [16]:
label_dim = len(np.unique(labels_train))
label_dim

2

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU 

In [18]:
# 调用函数，将这些 numpy 数组转换为 tensors
train_tensors = batch_dataframe_to_tensors(Methylation_train, miRNASeq_train, RNAseq_train)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_train = train_tensors[0]
miRNASeq_train = train_tensors[1]
RNAseq_train = train_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [19]:
# 调用函数，将这些 numpy 数组转换为 tensors
test_tensors = batch_dataframe_to_tensors(Methylation_test, miRNASeq_test, RNAseq_test)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_test = test_tensors[0]
miRNASeq_test = test_tensors[1]
RNAseq_test = test_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [20]:
# # 调用函数，将这些 numpy 数组转换为 tensors
# val_tensors = batch_dataframe_to_tensors(Methylation_val, miRNASeq_val, RNAseq_val)

# # 现在 tensors 是一个包含这些转换后张量的列表
# Methylation_val = val_tensors[0]
# miRNASeq_val = val_tensors[1]
# RNAseq_val = val_tensors[2]

In [21]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],

In [22]:
onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)

In [23]:
# 将DataFrame的值转换为Tensor
labels_te_tensor = torch.LongTensor(labels_test.values)
labels_te_tensor

tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],

In [24]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],

In [25]:
# # 将DataFrame的值转换为Tensor
# labels_val_tensor = torch.LongTensor(labels_val.values)
# labels_val_tensor

In [26]:
# 定义数据集
train_dataset = TensorDataset(Methylation_train, miRNASeq_train, RNAseq_train, labels_tr_tensor)
test_dataset = TensorDataset(Methylation_test, miRNASeq_test, RNAseq_test, labels_te_tensor)
# val_dataset = TensorDataset(Methylation_val, miRNASeq_val, RNAseq_val, labels_val_tensor)

# 定义 DataLoader，使用自定义采样器
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # 测试集一般不需要打乱顺序
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # 验证集也一般不需要重复采样

In [27]:
for i, batch in enumerate(train_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")


Batch 1: 153 samples
Batch 2: 153 samples
Batch 3: 152 samples


In [28]:
for i, batch in enumerate(test_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")

Batch 1: 152 samples


In [29]:
# for i, batch in enumerate(val_loader):
#     print(f"Batch {i + 1}: {len(batch[0])} samples")

In [30]:
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
from typing import Dict, Tuple, List
import gc

def plot_metrics(lr: float, depth: int, heads: int, dim_head: int, beta: float, adj_parameter: int, dropout: float) -> None:
    """生成高清训练指标图表"""
    
    # ================== 配置参数 ==================
    SAVE_DIR = '../../result/BRCA_survival/'
    FILE_FORMAT = 'pdf'  # 推荐矢量格式
    DPI = 600           # 印刷级分辨率
    FONT_CONFIG = {
        'font.family': 'DejaVu Sans',  # 跨平台兼容字体
        'font.size': 12,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'legend.fontsize': 10
    }
    STYLE_CONFIG = {
        'train': {'color': '#E74C3C', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},  # 红色
        'val': {'color': '#3498DB', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},    # 蓝色
        'test': {'color': '#2ECC71', 'ls': '-', 'lw': 2.5, 'alpha': 0.9}     # 绿色
    }
    
    # ================== 初始化设置 ==================
    rcParams.update(FONT_CONFIG)
    os.makedirs(SAVE_DIR, exist_ok=True)
    param_id = f"lr{lr:.0e}_depth{depth}heads{heads}dim_head{dim_head}adj_parameter{adj_parameter}dropout{dropout}beta{beta}".replace('.', '').replace('+', '')

    try:
        # ================== 创建画布 ==================
        fig = plt.figure(figsize=(18, 12), dpi=DPI, facecolor='white')
        fig.suptitle(f'Training Metrics ({param_id})', y=1.02, fontsize=16, fontweight='bold')
        
        # ================== 子图配置 ==================
        axes = [
            fig.add_subplot(221),  # Loss
            fig.add_subplot(222),  # Accuracy
            fig.add_subplot(223),  # F1 Score
            fig.add_subplot(224)   # AUC
        ]
        
        metric_data = [
            ('Loss', (train_losses, test_losses), 'log' if max(train_losses) > 1e3 else 'linear'),
            ('Accuracy', (train_accuracies, test_accuracies), 'linear'),
            ('F1 Score', (train_f1_scores, test_f1_scores), 'linear'),
            ('AUC', (train_auc_scores, test_auc_scores), 'linear')
        ]

        # ================== 绘制图表 ==================
        for ax, (title, (train, test), scale) in zip(axes, metric_data):
            # 绘制曲线
            ax.plot(train, label='Train', **STYLE_CONFIG['train'])
            # ax.plot(val, label='Val', **STYLE_CONFIG['val'])
            ax.plot(test, label='Test', **STYLE_CONFIG['test'])
            
            # 样式设置
            ax.set_title(title, pad=15, fontweight='semibold')
            ax.set_xlabel('Epochs', labelpad=10)
            ax.set_ylabel(title, labelpad=10)
            ax.grid(True, which='both', ls='--', alpha=0.4)
            ax.set_yscale(scale)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            # 添加图例
            if ax == axes[0]:  # 只在第一个子图添加图例
                handles, labels = ax.get_legend_handles_labels()
                fig.legend(handles, labels, loc='upper center', 
                          bbox_to_anchor=(0.5, 1.0), ncol=3, frameon=False)

        # ================== 布局优化 ==================
        plt.tight_layout(pad=3.0, w_pad=2.0, h_pad=2.0)
        
        # ================== 保存图像 ==================
        save_path = os.path.join(SAVE_DIR, f'metrics_{param_id}.{FILE_FORMAT}')
        plt.savefig(
            save_path,
            dpi=DPI,
            bbox_inches='tight',
            facecolor=fig.get_facecolor(),
            edgecolor='none',
            transparent=False,
            metadata={'CreationDate': None}  # 确保可重复性
        )
        
    except Exception as e:
        print(f"图表生成失败: {str(e)}")
    finally:
        # ================== 资源清理 ==================
        plt.close(fig)
        gc.collect()

In [31]:
def train_epoch(data_loader, model, optimizer, adj_parameter, output_attentions=False):
    model.train()
    total_loss = 0.0
    all_probs, all_labels = [], []

    for batch in data_loader:
        Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
            tensor.to(device) for tensor in batch
        ]

        # 生成三维邻接张量
        graph_train = np.stack([
            gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
        ], axis=-1)
        graph_train = torch.from_numpy(graph_train).float().to(device).permute(0, 2, 1)

        # 梯度更新流程
        optimizer.zero_grad()
        ci_loss, ci = model(
            Methylation_batch, 
            miRNASeq_batch,
            RNAseq_batch,
            graph_train,
            labels_batch,
            output_attentions=output_attentions
        )
        ci_loss.backward()
        optimizer.step()

        # 结果收集
        total_loss += ci_loss.item() * labels_batch.size(0)
        all_probs.append(F.softmax(ci, dim=1).detach().cpu().numpy())
        all_labels.append(labels_batch.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [32]:
# def val_epoch(data_loader, model, adj_parameter, output_attentions=False):
#     model.eval()
#     total_loss = 0.0
#     all_labels, all_probs = [], []

#     with torch.no_grad():
#         for batch in data_loader:
#             # 数据预处理
#             Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
#                 x.to(device) for x in batch
#             ]
#             # print(miRNASeq_batch.shape) #torch.Size([32, 217])
#             # 生成三维邻接张量
#             graph_val = torch.from_numpy(np.stack([
#                 gen_trte_adj_mat(d.cpu(), adj_parameter) 
#                 for d in [Methylation_batch, miRNASeq_batch, RNAseq_batch]
#             ], axis=-1)).float().to(device).permute(0,2,1)

#             # 前向计算
#             ci_loss, ci = model(
#                 Methylation_batch, 
#                 miRNASeq_batch,
#                 RNAseq_batch,
#                 graph_val,
#                 labels_batch,
#                 output_attentions=output_attentions
#             )

#             # 累计结果
#             total_loss += ci_loss.item() * labels_batch.size(0)
#             all_probs.append(F.softmax(ci, 1).detach().cpu().numpy())
#             all_labels.append(labels_batch.detach().cpu().numpy().flatten())

#     return (
#         total_loss / len(data_loader.dataset),
#         np.concatenate(all_probs),
#         np.concatenate(all_labels)
#     )

In [33]:
def test_epoch(data_loader, model, adj_parameter, output_attentions=False):
    model.eval()
    total_loss = 0.0
    all_probs, all_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            # 数据预处理 (保持原始变量名)
            Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_test_tensor = [
                tensor.to(device) for tensor in batch
            ]

            # 生成邻接矩阵 (保持原始逻辑但更紧凑)
            graph_test = torch.from_numpy(
                np.stack([
                    gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
                ], axis=-1)
            ).float().to(device).permute(0, 2, 1)

            # 前向计算
            loss, logits = model(
                Methylation_batch,
                miRNASeq_batch,
                RNAseq_batch,
                graph_test,
                labels_test_tensor,
                output_attentions=output_attentions
            )

            # 结果收集
            total_loss += loss.item() * labels_test_tensor.size(0)
            all_probs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
            all_labels.append(labels_test_tensor.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [34]:
# 用于保存指标数据的列表
train_losses = []
val_losses = []
test_losses = []
val_accuracies = []
val_f1_scores = []
val_auc_scores = []
test_accuracies = []
test_f1_scores = []
test_auc_scores = []
train_accuracies = []
train_f1_scores = []
train_auc_scores = []

def train_test(
        Methylation_train,
        miRNASeq_train,
        RNAseq_train,
        lr,
        depth, 
        heads, 
        dim_head,
        beta, 
        attn_dropout,
        ff_dropout, 
        classifier_dropout,
        adj_parameter,
        num_epoch_pretrain
):
    # 初始化最佳指标跟踪器
    best_metrics = {
        'epoch': -1,
        'test_acc': 0.0,
        'test_f1': 0.0,
        'test_auc': 0.0,
    }
    
    DEPTH = depth
    HEAD = heads
    HEAD_DIM = dim_head

    classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
    classifier_dim = [
        [classifier_input[0], 300, 200],
        [classifier_input[1], 300, 200],
        [classifier_input[2], 300, 200]
    ]
    col_dim = 32

    # 确定 row_dim 和 embeding_num
    if num_view <= 2:
        embeding = True
        embeding_num = 32
        row_dim = embeding_num  # 请根据需要修改此处
    else:
        embeding = False
        row_dim = num_view
        embeding_num = 32
    
    model = pathformer_model(
        row_dim=row_dim,
        col_dim=col_dim,
        depth=DEPTH,
        heads=HEAD,
        dim_head=HEAD_DIM,
        classifier_input=classifier_input,
        classifier_dim=classifier_dim,
        label_dim=label_dim,
        embeding=embeding,
        embeding_num=embeding_num,
        beta=beta,
        attn_dropout=attn_dropout,
        ff_dropout=ff_dropout,
        classifier_dropout=classifier_dropout,
        num_view=num_view,
        dim_hvcdn=dim_hvcdn
    )

    if torch.cuda.is_available():
        model.to(device)

    logging.info("\nPretrain GCNs...")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    logging.info("\nTraining...")
    for epoch in range(num_epoch_pretrain + 1):
        # 每一轮都进行训练
        train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, output_attentions=False)
        train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
        train_f1 = f1_score(labels_train_tensor, train_probs.argmax(1))
        train_auc = roc_auc_score(labels_train_tensor, train_probs[:, 1])

        logging.info(f"Train Epoch {epoch}, Loss: {train_loss}")
        logging.info(f"Train Epoch {epoch}, ACC: Train ACC: {train_acc:.3f}")
        logging.info(f"Train Epoch {epoch}, f1: Train F1: {train_f1:.3f}")
        logging.info(f"Train Epoch {epoch}, AUC: Train AUC: {train_auc:.3f}")
        
        # 每一轮都进行验证
        # val_loss, val_probs, labels_validation_tensor = val_epoch(val_loader, model, adj_parameter, output_attentions=False)
        # val_acc = accuracy_score(labels_validation_tensor, val_probs.argmax(1))
        # val_f1 = f1_score(labels_validation_tensor, val_probs.argmax(1))
        # val_auc = roc_auc_score(labels_validation_tensor, val_probs[:, 1])
        # logging.info(f"val Epoch {epoch}, Loss: {val_loss}")
        # logging.info(f"val Epoch {epoch}, ACC: val ACC: {val_acc:.3f}")
        # logging.info(f"val Epoch {epoch}, f1: val F1: {val_f1:.3f}")
        # logging.info(f"val Epoch {epoch}, AUC: val AUC: {val_auc:.3f}")
        
        # 每一轮都进行测试
        test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, output_attentions=False)
        test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
        test_f1 = f1_score(labels_test_tensor, test_probs.argmax(1))
        test_auc = roc_auc_score(labels_test_tensor, test_probs[:, 1])
        logging.info(f"test Epoch {epoch}, Loss: {test_loss}")
        logging.info(f"test Epoch {epoch}, ACC: test ACC: {test_acc:.3f}")
        logging.info(f"test Epoch {epoch}, f1: test F1: {test_f1:.3f}")
        logging.info(f"test Epoch {epoch}, AUC: test AUC: {test_auc:.3f}")

        # 更新最佳指标 (以验证AUC为选择标准)
        if test_auc > best_metrics['test_auc'] or \
           (test_auc == best_metrics['test_auc'] and test_f1 > best_metrics['test_f1']):
            best_metrics.update({
                'epoch': epoch,
                'test_acc': test_acc,
                'test_f1': test_f1,
                'test_auc': test_auc
            })

        logging.info("\n")

        # 保存训练指标
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        train_f1_scores.append(train_f1)
        train_auc_scores.append(train_auc)
        
        # val_losses.append(val_loss)
        # val_accuracies.append(val_acc)
        # val_f1_scores.append(val_f1)
        # val_auc_scores.append(val_auc)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)
        test_f1_scores.append(test_f1)
        test_auc_scores.append(test_auc)

        # 绘制并保存指标图
        plot_metrics(lr, depth, heads, dim_head, beta, adj_parameter, dropout=attn_dropout)

    return (
        best_metrics['test_acc'],
        best_metrics['test_f1'],
        best_metrics['test_auc']
    )


In [35]:
# # 用于保存指标数据的列表
# train_losses = []
# val_losses = []
# test_losses = []
# val_accuracies = []
# val_f1_scores = []
# val_auc_scores = []
# test_accuracies = []
# test_f1_scores = []
# test_auc_scores = []
# train_accuracies = []
# train_f1_scores = []
# train_auc_scores = []
# train_c_index = []
# val_c_index = []
# test_c_index = []

# def train_test(
#         Methylation_train,
#         Methylation_test,
#         miRNASeq_train,
#         miRNASeq_test,
#         RNAseq_train,
#         RNAseq_test,
#         label_train,
#         label_test, 
#         labels_tr_tensor,
#         lr,
#         depth, 
#         heads, 
#         dim_head,
#         beta, 
#         attn_dropout,
#         ff_dropout, 
#         classifier_dropout,
#         adj_parameter,
#         num_epoch_pretrain
# ):
    
#     DEPTH = depth
#     HEAD = heads
#     HEAD_DIM = dim_head

#     classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
#     classifier_dim = [
#         [classifier_input[0], 300, 200],
#         [classifier_input[1], 300, 200],
#         [classifier_input[2], 300, 200]
#     ]
#     col_dim = 71

#     # 确定 row_dim 和 embeding_num
#     if num_view <= 2:
#         embeding = True
#         embeding_num = 71
#         row_dim = embeding_num  # 请根据需要修改此处
#     else:
#         embeding = False
#         row_dim = num_view
#         embeding_num = 71
    
#     model = pathformer_model(
#         row_dim=row_dim,
#         col_dim=col_dim,
#         depth=DEPTH,
#         heads=HEAD,
#         dim_head=HEAD_DIM,
#         classifier_input=classifier_input,
#         classifier_dim=classifier_dim,
#         label_dim=label_dim,
#         embeding=embeding,
#         embeding_num=embeding_num,
#         beta=beta,
#         attn_dropout=attn_dropout,
#         ff_dropout=ff_dropout,
#         classifier_dropout=classifier_dropout,
#         num_view=num_view,
#         dim_hvcdn=dim_hvcdn
#     )

#     if torch.cuda.is_available():
#         model.to(device)

#     print("\nPretrain GCNs...")
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#     print("\nTraining...")
#     for epoch in range(num_epoch_pretrain + 1):
#         # 每一轮都进行训练
#         train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, epoch, output_attentions=False)
#         train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
#         train_f1 = f1_score(labels_train_tensor, train_probs.argmax(1))
#         train_auc = roc_auc_score(labels_train_tensor, train_probs[:, 1])
# #         print(f"Train Epoch {epoch}, Loss: {train_loss}")
# #         print(f"Train Epoch {epoch}, ACC: Train ACC: {train_acc:.3f}")
# #         print(f"Train Epoch {epoch}, f1: Train F1: {train_f1:.3f}")
# #         print(f"Train Epoch {epoch}, AUC: Train AUC: {train_auc:.3f}")
        
#         # 每一轮都进行验证
#         val_loss, val_probs, labels_val_tensor = val_epoch(val_loader, model, adj_parameter, epoch, output_attentions=False)
#         val_acc = accuracy_score(labels_val_tensor, val_probs.argmax(1))
#         val_f1 = f1_score(labels_val_tensor, val_probs.argmax(1))
#         val_auc = roc_auc_score(labels_val_tensor, val_probs[:, 1])
# #         print(f"val Epoch {epoch}, Loss: {val_loss}")
# #         print(f"val Epoch {epoch}, ACC: val ACC: {val_acc:.3f}")
# #         print(f"val Epoch {epoch}, f1: val F1: {val_f1:.3f}")
# #         print(f"val Epoch {epoch}, AUC: val AUC: {val_auc:.3f}")
        
#         # 每一轮都进行测试
#         test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, epoch, output_attentions=False)
#         test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
#         test_f1 = f1_score(labels_test_tensor, test_probs.argmax(1))
#         test_auc = roc_auc_score(labels_test_tensor, test_probs[:, 1])
# #         print(f"test Epoch {epoch}, Loss: {test_loss}")
# #         print(f"test Epoch {epoch}, ACC: test ACC: {test_acc:.3f}")
# #         print(f"test Epoch {epoch}, f1: test F1: {test_f1:.3f}")
# #         print(f"test Epoch {epoch}, AUC: test AUC: {test_auc:.3f}")

#         print("\n")

#         # 保存训练指标
#         train_losses.append(train_loss)
#         train_accuracies.append(train_acc)
#         train_f1_scores.append(train_f1)
#         train_auc_scores.append(train_auc)
        
#         val_losses.append(val_loss)
#         val_accuracies.append(val_acc)
#         val_f1_scores.append(val_f1)
#         val_auc_scores.append(val_auc)
        
#         test_losses.append(test_loss)
#         test_accuracies.append(test_acc)
#         test_f1_scores.append(test_f1)
#         test_auc_scores.append(test_auc)

#         # 绘制并保存指标图
#         plot_metrics(lr, depth, heads, dim_head, beta, dropout=attn_dropout)

#     # 返回最终的预测结果和指标
#     final_predictions = test_probs.argmax(1)
#     return final_predictions, test_acc, test_f1, test_auc


In [36]:
import logging
import os
import time
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# 配置日志和路径
LOG_FILE = '../../result/BRCA_survival/training_log_CMFM.txt'
RESULT_FILE = '../../result/BRCA_survival/mogat_training_results.xlsx'
os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    filemode='w'  # 每次运行覆盖旧日志
)

# 超参数配置（便于维护和修改）
HP_CONFIG = {
    'lr': [0.0001, 0.001, 0.01],
    'depth': [2, 3, 4, 6],
    'heads': [4, 6, 8],
    'dim_head': [12, 16, 32, 64],
    'dropout': [0.3],
    'beta': [1],
    'adj_parameter': [2, 4, 6, 8, 10]
}

# 计算总组合数
total_combinations = (
    len(HP_CONFIG['lr']) *
    len(HP_CONFIG['depth']) *
    len(HP_CONFIG['heads']) *
    len(HP_CONFIG['dim_head']) *
    len(HP_CONFIG['dropout']) *
    len(HP_CONFIG['beta']) *
    len(HP_CONFIG['adj_parameter'])
)

# 结果缓存配置
BATCH_SIZE = 5  # 每5个结果写入一次文件
results_cache = []

def save_results(batch: list, force_save: bool = False):
    """批量保存结果到文件"""
    if not force_save and len(batch) < BATCH_SIZE:
        return
    
    try:
        df = pd.DataFrame(batch)
        header = not os.path.exists(RESULT_FILE)
        
        with pd.ExcelWriter(
            RESULT_FILE,
            mode='a' if os.path.exists(RESULT_FILE) else 'w',
            engine='openpyxl',
            if_sheet_exists='overlay' if os.path.exists(RESULT_FILE) else None
        ) as writer:
            df.to_excel(
                writer,
                index=False,
                sheet_name='Results',
                header=header,
                startrow=writer.sheets['Results'].max_row if not header else 0
            )
        batch.clear()
    except Exception as e:
        logging.error(f"保存结果失败: {str(e)}")
        raise

# 主训练循环
current_combination = 0
start_total = time.time()

try:
    for lr in HP_CONFIG['lr']:
        for depth in HP_CONFIG['depth']:
            for heads in HP_CONFIG['heads']:
                for dim_head in HP_CONFIG['dim_head']:
                    for dropout in HP_CONFIG['dropout']:
                        for beta in HP_CONFIG['beta']:
                            for adj_param in HP_CONFIG['adj_parameter']:
                                current_combination += 1
                                start_iter = time.time()
                                
                                # 日志记录当前参数
                                param_str = (
                                    f"lr={lr}, depth={depth}, heads={heads}, "
                                    f"dim_head={dim_head}, dropout={dropout}, "
                                    f"beta={beta}, adj_parameter={adj_param} "
                                    f"({current_combination}/{total_combinations})"
                                )
                                logging.info(f"开始训练: {param_str}")

                                try:
                                    # 执行训练
                                    acc, f1, auc = train_test(
                                        Methylation_train=Methylation_train,
                                        miRNASeq_train=miRNASeq_train,
                                        RNAseq_train=RNAseq_train,
                                        lr=lr,
                                        depth=depth,
                                        heads=heads,
                                        dim_head=dim_head,
                                        beta=beta,
                                        attn_dropout=dropout,
                                        ff_dropout=dropout,
                                        classifier_dropout=dropout,
                                        adj_parameter=adj_param,
                                        num_epoch_pretrain=1500
                                    )

                                    # 缓存结果
                                    results_cache.append({
                                        'lr': lr,
                                        'depth': depth,
                                        'heads': heads,
                                        'dim_head': dim_head,
                                        'dropout': dropout,
                                        'beta': beta,
                                        'adj_parameter': adj_param,
                                        'accuracy': acc,
                                        'f1_score': f1,
                                        'auc': auc
                                    })

                                    # 定期保存
                                    save_results(results_cache)

                                    # 记录成功信息
                                    elapsed = time.time() - start_iter
                                    logging.info(
                                        f"完成: {param_str} | "
                                        f"耗时: {elapsed:.1f}s | "
                                        f"结果: ACC={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}"
                                    )

                                except torch.cuda.OutOfMemoryError:
                                    logging.warning(f"显存不足，跳过组合: {param_str}")
                                    torch.cuda.empty_cache()
                                except Exception as e:
                                    logging.error(f"训练失败: {param_str} | 错误: {str(e)}")
                                    continue
                                # 用于保存指标数据的列表
                                train_losses = []
                                val_losses = []
                                test_losses = []
                                val_accuracies = []
                                val_f1_scores = []
                                val_auc_scores = []
                                test_accuracies = []
                                test_f1_scores = []
                                test_auc_scores = []
                                train_accuracies = []
                                train_f1_scores = []
                                train_auc_scores = []

    # 最后强制保存剩余结果
    save_results(results_cache, force_save=True)

except KeyboardInterrupt:
    logging.warning("用户中断训练，保存已完成的实验结果...")
    save_results(results_cache, force_save=True)
finally:
    total_time = time.time() - start_total
    logging.info(f"全部完成! 总耗时: {total_time/3600:.2f} 小时")

print("实验完成，结果保存在:", RESULT_FILE)

In [ ]:
# import os
# import time
# import pandas as pd
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # 创建一个 DataFrame 用于存储结果
# results_df = pd.DataFrame(columns=['lr', 'depth', 'heads', 'dim_head', 'dropout', 'beta', 'accuracy', 'f1_score', 'auc'])

# # 设置单一的超参数组合
# lr = 0.0001  # 你想要的学习率
# depth = 6  # 你想要的层数
# heads = 4  # 你想要的注意力头数
# dim_head = 32  # 你想要的注意力头的维度
# dropout = 0.5  # 你想要的 dropout 比例
# beta = 1  # 你想要的 beta 值

# # 训练参数组合
# start_time = time.time()  # 记录开始时间
# print(f"训练参数：lr={lr}, depth={depth}, heads={heads}, "
#       f"dim_head={dim_head}, dropout={dropout}, beta={beta}")

# # 训练函数假设已定义，返回最后的预测和各项指标
# try:
#     last_predictions, acc, f1, auc = train_test(
#         Methylation_train=Methylation_train,
#         Methylation_test=Methylation_test,
#         miRNASeq_train=miRNASeq_train,
#         miRNASeq_test=miRNASeq_test,
#         RNAseq_train=RNAseq_train,
#         RNAseq_test=RNAseq_test,
#         label_train=labels_train,
#         label_test=labels_test,
#         labels_tr_tensor=labels_tr_tensor,
#         lr=lr,
#         depth=depth,
#         heads=heads,
#         dim_head=dim_head,
#         beta=beta,
#         attn_dropout=dropout,
#         ff_dropout=dropout,
#         classifier_dropout=dropout,
#         num_epoch_pretrain=2000,
#     )

#     # 清空列表以便下一个超参数组合使用
#     train_losses = []
#     val_losses = []
#     test_losses = []
    
#     train_accuracies = []
#     val_accuracies = []
#     test_accuracies = []
    
#     train_f1_scores = []
#     val_f1_scores = []
#     test_f1_scores = []
    
#     train_auc_scores = []
#     val_auc_scores = []
#     test_auc_scores = []
    
#     # 检查最后的预测结果是否全部一致
#     if last_predictions is not None:
#         last_predictions_tensor = torch.tensor(last_predictions)  # 转换为张量
#         if not torch.all(last_predictions_tensor == last_predictions_tensor[0]):
#             print("最后一轮的预测结果不一致，保存该参数组合和评价指标。")
            
#             # 保存结果到 DataFrame
#             result = {
#                 'lr': lr,
#                 'depth': depth,
#                 'heads': heads,
#                 'dim_head': dim_head,
#                 'dropout': dropout,
#                 'beta': beta,
#                 'accuracy': acc,
#                 'f1_score': f1,
#                 'auc': auc
#             }

#             result_df = pd.DataFrame([result])

#             # 确保 result_df 不为空且不全为 NA
#             if result_df.notnull().any().any():
#                 results_df = pd.concat([results_df, result_df], ignore_index=True)

#                 # 检查 Excel 文件是否存在，如果不存在则创建
#                 if not os.path.isfile('../../result/BRCA_survival/mogat_training_results.xlsx'):
#                     results_df.to_excel('../../result/BRCA_survival/mogat_training_results.xlsx', index=False, sheet_name='Results')
#                 else:
#                     # 立即写入 Excel 文件
#                     with pd.ExcelWriter('../../result/BRCA_survival/mogat_training_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
#                         results_df.to_excel(writer, index=False, sheet_name='Results', startrow=writer.sheets['Results'].max_row)
#         else:
#             print("最后一轮的预测结果全部一致，不保存该参数组合。")

# except Exception as e:
#     print(f"训练参数组合 lr={lr}, depth={depth}, heads={heads}, dim_head={dim_head}, dropout={dropout}, beta={beta} 时发生错误: {e}")

# elapsed_time = time.time() - start_time
# print(f"训练完成，耗时: {elapsed_time:.2f} 秒")

# # 最终保存（如果需要）
# if not results_df.empty:
#     results_df.to_excel('../../result/BRCA_survival/mogat_training_results.xlsx', index=False, sheet_name='Results')
#     print("参数组合和评价指标已保存为 mogat_training_results.xlsx 文件。")
# else:
#     print("没有符合条件的参数组合，未保存任何结果。")
